# QuASAr Theoretical Benchmark Sweeps

This notebook mirrors the execution sweeps from `quasar_benchmark_sweeps.ipynb` using the theoretical
cost estimation utilities to predict runtime and memory footprints. Adjust the configuration below to
explore different circuit families and planner settings.


In [ ]:
from __future__ import annotations

import json
import logging
from dataclasses import replace
from typing import Any, Dict, Iterable, List, Optional

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from IPython.display import display

from benchmarks.unified import generate_benchmark_circuit
from quasar.planner import PlannerConfig
from quasar.theoretical import (
    TheoreticalAblationOptions,
    analyze_without_disjoint,
    estimate_decision_diagram,
    estimate_quasar,
    estimate_statevector,
    estimate_tableau,
)

LOGGER = logging.getLogger("quasar.theoretical_sweeps")
if not LOGGER.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("[%(asctime)s] %(levelname)s %(message)s"))
    LOGGER.addHandler(handler)
LOGGER.propagate = False
LOGGER.setLevel(logging.INFO)

pd.options.display.float_format = "{:.2f}".format
sns.set_theme(style="whitegrid")

def _aggregate_work(units: Dict[str, float]) -> float:
    return float(sum(units.values())) if units else 0.0


In [ ]:
def _make_planner_cfg(overrides: Optional[Dict[str, Any]] = None, *, force_full_search: bool = False) -> PlannerConfig:
    params = dict(overrides or {})
    if force_full_search:
        params.setdefault("quick_path_partition_threshold", -1)
        params.setdefault("quick_path_gate_threshold", -1)
        params.setdefault("quick_path_qubit_threshold", -1)
    return PlannerConfig(**params)

def sweep_theoretical_quasar(
    *,
    family: str,
    varying: str,
    values: Iterable[int],
    fixed_qubits: int,
    fixed_depth: int,
    circuit_kwargs: Optional[Dict[str, Any]] = None,
    planner_overrides: Optional[Dict[str, Any]] = None,
    analysis_fn=None,
    force_full_search: bool = False,
    ablation: Optional[TheoreticalAblationOptions] = None,
) -> List[Dict[str, Any]]:
    records: List[Dict[str, Any]] = []
    for value in values:
        if varying == "qubits":
            num_qubits = int(value)
            depth = int(fixed_depth)
        elif varying == "depth":
            num_qubits = int(fixed_qubits)
            depth = int(value)
        else:
            raise ValueError("varying must be 'qubits' or 'depth'")
        circuit = generate_benchmark_circuit(
            num_qubits=num_qubits,
            depth=depth,
            family=family,
            **(circuit_kwargs or {}),
        )
        cfg = _make_planner_cfg(planner_overrides, force_full_search=force_full_search)
        ablation_opts = ablation
        if force_full_search:
            if ablation_opts is None:
                ablation_opts = TheoreticalAblationOptions(force_full_planner_search=True)
            elif not ablation_opts.force_full_planner_search:
                ablation_opts = replace(ablation_opts, force_full_planner_search=True)
        estimate = estimate_quasar(
            circuit,
            planner_cfg=cfg,
            analysis_fn=analysis_fn,
            ablation=ablation_opts,
        )
        record = {
            "backend": "quasar",
            "family": family,
            "varying": varying,
            "value": int(value),
            "num_qubits": num_qubits,
            "depth": depth,
            "work_units_total": _aggregate_work(estimate.work_units_by_label),
            "work_units_by_label": dict(estimate.work_units_by_label),
            "peak_memory_bytes": int(estimate.peak_memory_bytes),
            "time_seconds": estimate.total_time_seconds,
            "ok": bool(estimate.ok),
            "single_backend": estimate.single_backend,
            "single_backend_reason": estimate.single_backend_reason,
            "plan_cost_units": float(estimate.plan_cost_units),
        }
        records.append(record)
    return records

def sweep_theoretical_baseline(
    *,
    backend: str,
    family: str,
    varying: str,
    values: Iterable[int],
    fixed_qubits: int,
    fixed_depth: int,
    circuit_kwargs: Optional[Dict[str, Any]] = None,
) -> List[Dict[str, Any]]:
    records: List[Dict[str, Any]] = []
    estimators = {
        "sv": estimate_statevector,
        "dd": estimate_decision_diagram,
        "tableau": estimate_tableau,
    }
    estimator = estimators.get(backend)
    if estimator is None:
        raise ValueError(f"Unsupported baseline backend '{backend}'")
    for value in values:
        if varying == "qubits":
            num_qubits = int(value)
            depth = int(fixed_depth)
        elif varying == "depth":
            num_qubits = int(fixed_qubits)
            depth = int(value)
        else:
            raise ValueError("varying must be 'qubits' or 'depth'")
        circuit = generate_benchmark_circuit(
            num_qubits=num_qubits,
            depth=depth,
            family=family,
            **(circuit_kwargs or {}),
        )
        estimate = estimator(circuit)
        record = {
            "backend": backend,
            "family": family,
            "varying": varying,
            "value": int(value),
            "num_qubits": num_qubits,
            "depth": depth,
            "work_units_total": float(estimate.work_units),
            "work_units_by_label": {estimate.work_unit_label: float(estimate.work_units)},
            "peak_memory_bytes": int(estimate.memory_bytes),
            "time_seconds": estimate.time_seconds,
            "ok": bool(estimate.ok),
            "reason": estimate.reason,
        }
        records.append(record)
    return records

def _to_dataframe(records: List[Dict[str, Any]], *, label: Optional[str] = None) -> pd.DataFrame:
    if not records:
        return pd.DataFrame()
    rows = []
    for rec in records:
        row = dict(rec)
        row["work_units_by_label"] = json.dumps(rec.get("work_units_by_label", {}), sort_keys=True)
        if label is not None:
            row["label"] = label
        rows.append(row)
    return pd.DataFrame(rows)


## Default sweep configuration
Adjust the values below to explore other circuit families, qubit ranges, or depths.


In [ ]:
family = "mixed"
qubit_sweep = [6, 8, 10, 12, 14, 16]
depth_sweep = [8, 12, 16, 20]

circuit_options = {"block_size": 4, "cutoff": 0.75, "tail_type": "sparse", "seed": 7}
planner_options = {"max_ram_gb": 4.0, "prefer_dd": True}


## QuASAr theoretical sweeps


In [ ]:
quasar_qubit_results = sweep_theoretical_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
)
quasar_depth_results = sweep_theoretical_quasar(
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
)
quasar_qubit_df = _to_dataframe(quasar_qubit_results, label="QuASAr (qubits)")
quasar_depth_df = _to_dataframe(quasar_depth_results, label="QuASAr (depth)")
display(quasar_qubit_df)
display(quasar_depth_df)


## Tableau baseline sweeps


In [ ]:
tableau_qubit_results = sweep_theoretical_baseline(
    backend="tableau",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
)
tableau_qubit_df = _to_dataframe(tableau_qubit_results, label="Tableau (qubits)")
display(tableau_qubit_df)


## Statevector baseline sweeps


In [ ]:
statevector_qubit_results = sweep_theoretical_baseline(
    backend="sv",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
)
statevector_qubit_df = _to_dataframe(statevector_qubit_results, label="Statevector (qubits)")
display(statevector_qubit_df)

statevector_depth_results = sweep_theoretical_baseline(
    backend="sv",
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
)
statevector_depth_df = _to_dataframe(statevector_depth_results, label="Statevector (depth)")
display(statevector_depth_df)


## Decision-diagram baseline sweeps


In [ ]:
dd_qubit_results = sweep_theoretical_baseline(
    backend="dd",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
)
dd_qubit_df = _to_dataframe(dd_qubit_results, label="Decision diagram (qubits)")
display(dd_qubit_df)


## Ablation: disable disjoint parallelisation


In [ ]:
ablation_disjoint = sweep_theoretical_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    ablation=TheoreticalAblationOptions(disable_disjoint_subcircuits=True),
)
ablation_disjoint_df = _to_dataframe(ablation_disjoint, label="QuASAr (no disjoint)")
display(ablation_disjoint_df)


## Ablation: disable method-based partitioning


In [ ]:
ablation_method = sweep_theoretical_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    ablation=TheoreticalAblationOptions(disable_method_partitioning=True),
)
ablation_method_df = _to_dataframe(ablation_method, label="QuASAr (no hybrid tail)")
display(ablation_method_df)


## Ablation: disable quick-path method selection


In [ ]:
ablation_quick_path = sweep_theoretical_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    force_full_search=True,
    ablation=TheoreticalAblationOptions(force_full_planner_search=True),
)
ablation_quick_path_df = _to_dataframe(ablation_quick_path, label="QuASAr (full search)")
display(ablation_quick_path_df)


## Seaborn summary plots
The plots below aggregate the main sweep results using the Seaborn styling configured above.


In [ ]:
qubit_sources = [
    ("quasar_qubit_df", "QuASAr full planner"),
    ("statevector_qubit_df", "Statevector baseline"),
    ("dd_qubit_df", "Decision diagram baseline"),
    ("tableau_qubit_df", "Tableau baseline"),
    ("ablation_disjoint_df", "QuASAr (no disjoint)"),
    ("ablation_method_df", "QuASAr (no hybrid tail)"),
    ("ablation_quick_path_df", "QuASAr (full search)"),
]
qubit_plot_frames: List[pd.DataFrame] = []
for name, label in qubit_sources:
    frame = locals().get(name)
    if frame is not None and not frame.empty:
        data = frame.copy()
        data["label"] = label
        data["sweep"] = "qubits"
        qubit_plot_frames.append(data)
qubit_plot_df = pd.concat(qubit_plot_frames, ignore_index=True) if qubit_plot_frames else pd.DataFrame()

if not qubit_plot_df.empty:
    qubit_plot_df = qubit_plot_df[qubit_plot_df.get("ok", True).astype(bool)]
    qubit_plot_df = qubit_plot_df.copy()
    qubit_plot_df["peak_memory_gib"] = qubit_plot_df["peak_memory_bytes"] / float(1024 ** 3)
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    sns.lineplot(
        data=qubit_plot_df,
        x="value",
        y="work_units_total",
        hue="label",
        style="label",
        markers=True,
        ax=axes[0],
    )
    axes[0].set_xlabel("Qubits")
    axes[0].set_ylabel("Work units (model)")
    axes[0].set_title("Theoretical work across variants")

    sns.lineplot(
        data=qubit_plot_df,
        x="value",
        y="peak_memory_gib",
        hue="label",
        style="label",
        markers=True,
        legend=False,
        ax=axes[1],
    )
    axes[1].set_xlabel("Qubits")
    axes[1].set_ylabel("Peak memory (GiB)")
    axes[1].set_title("Peak memory estimates")

    speedup_frames: List[pd.DataFrame] = []
    baseline = locals().get("statevector_qubit_df")
    if baseline is not None and not baseline.empty:
        baseline_cols = ["family", "varying", "value", "num_qubits", "depth"]
        baseline_subset = baseline[baseline_cols + ["work_units_total"]].rename(
            columns={"work_units_total": "baseline_work_units"}
        )
        for variant_name, label in [
            ("quasar_qubit_df", "QuASAr full planner"),
            ("ablation_disjoint_df", "QuASAr (no disjoint)"),
            ("ablation_method_df", "QuASAr (no hybrid tail)"),
            ("ablation_quick_path_df", "QuASAr (full search)"),
        ]:
            variant = locals().get(variant_name)
            if variant is None or variant.empty:
                continue
            merged = variant.merge(baseline_subset, on=baseline_cols, how="inner")
            if merged.empty:
                continue
            merged = merged.copy()
            merged["speedup_vs_statevector"] = merged["baseline_work_units"] / merged["work_units_total"]
            merged["label"] = label
            speedup_frames.append(merged[["value", "label", "speedup_vs_statevector"]])
        speedup_df = pd.concat(speedup_frames, ignore_index=True) if speedup_frames else pd.DataFrame()
    else:
        speedup_df = pd.DataFrame()

    if not speedup_df.empty:
        sns.lineplot(
            data=speedup_df,
            x="value",
            y="speedup_vs_statevector",
            hue="label",
            style="label",
            markers=True,
            legend=False,
            ax=axes[2],
        )
        axes[2].axhline(1.0, color="#666666", linestyle="--", linewidth=1)
        axes[2].set_xlabel("Qubits")
        axes[2].set_ylabel("Speedup vs statevector (×)")
        axes[2].set_title("Relative work vs baseline")
    else:
        axes[2].set_visible(False)

    handles, labels = axes[0].get_legend_handles_labels()
    axes[0].legend(handles, labels, title="Variant", bbox_to_anchor=(1.05, 1), loc="upper left")
    axes[1].get_legend().remove() if axes[1].get_legend() else None
    plt.tight_layout()
else:
    print("No qubit data available for plotting.")
